#### **Training an ANN Model on Rice Type Classification**

In [48]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset , DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [49]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [50]:
df = pd.read_csv("E:\\Notes\\Python'\\PyTorch\\Datasets\\riceClassification.csv")
df.head()

,id,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,1,4537,92.229316,64.012769,0.719916,4677,76.004525,0.657536,273.085,0.764510,1.440796,1
1,2,2872,74.691881,51.400454,0.725553,3015,60.471018,0.713009,208.317,0.831658,1.453137,1
2,3,3048,76.293164,52.043491,0.731211,3132,62.296341,0.759153,210.012,0.868434,1.465950,1
3,4,3073,77.033628,51.928487,0.738639,3157,62.551300,0.783529,210.657,0.870203,1.483456,1
4,5,3693,85.124785,56.374021,0.749282,3802,68.571668,0.769375,230.332,0.874743,1.510000,1


In [51]:
df.isna().sum()
df.drop(columns=["id"], axis=1, inplace=True)
df.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,4537,92.229316,64.012769,0.719916,4677,76.004525,0.657536,273.085,0.764510,1.440796,1
1,2872,74.691881,51.400454,0.725553,3015,60.471018,0.713009,208.317,0.831658,1.453137,1
2,3048,76.293164,52.043491,0.731211,3132,62.296341,0.759153,210.012,0.868434,1.465950,1
3,3073,77.033628,51.928487,0.738639,3157,62.551300,0.783529,210.657,0.870203,1.483456,1
4,3693,85.124785,56.374021,0.749282,3802,68.571668,0.769375,230.332,0.874743,1.510000,1


In [52]:
df["Class"].unique()

array([1, 0])

In [53]:
original_df = df.copy()

for column in df.columns:
    df[column] = df[column]/df[column].abs().max()  # Normalizing

df.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,0.444368,0.503404,0.775435,0.744658,0.424873,0.666610,0.741661,0.537029,0.844997,0.368316,1.0
1,0.281293,0.407681,0.622653,0.750489,0.273892,0.530370,0.804230,0.409661,0.919215,0.371471,1.0
2,0.298531,0.416421,0.630442,0.756341,0.284520,0.546380,0.856278,0.412994,0.959862,0.374747,1.0
3,0.300979,0.420463,0.629049,0.764024,0.286791,0.548616,0.883772,0.414262,0.961818,0.379222,1.0
4,0.361704,0.464626,0.682901,0.775033,0.345385,0.601418,0.867808,0.452954,0.966836,0.386007,1.0


In [54]:
X = df.iloc[:,:-1].values  # .values -> converts pandas to numpy df which is then passed to dataset
y = df.iloc[:, -1].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5)

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(12729, 10)
(2728, 10)
(2728, 10)


In [55]:
# Custom Dataset

class CustomDataset(Dataset):

    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32).to(device)
        self.labels = torch.tensor(labels, dtype=torch.long).to(device)

    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, index):
        return self.features[index], self.labels[index]


In [65]:
training_data = CustomDataset(X_train, y_train)
validation_data = CustomDataset(X_val, y_val)
test_data = CustomDataset(X_test, y_test)

In [66]:
train_loader = DataLoader(training_data, batch_size=32, shuffle=True)
val_loader = DataLoader(validation_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

In [58]:
# Model Architecture

class NN(nn.Module):

    def __init__(self, num_features):

        super().__init__()

        self.model = nn.Sequential(
            nn.Linear(num_features, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(p=0.3),
            nn.Linear(64,10),
            nn.BatchNorm1d(10),
            nn.ReLU(),
            nn.Dropout(p=0.3),
            nn.Linear(10,2)
        )

    def forward(self,x):
        return self.model(x)


In [59]:
Epochs = 100
Learning_rate=1e-3

In [60]:
X_train.shape[1]

10

In [61]:
model = NN(X_train.shape[1])
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=Learning_rate)

In [72]:
# Training Loop 

for epoch in range(Epochs):

    model.train()
    running_train_loss=0.0
    correct=0.0
    total = 0.0

    for batch_features, batch_labels in train_loader:

        batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

        optimizer.zero_grad()
        output = model(batch_features)
        loss = criterion(output, batch_labels)
        loss.backward()
        optimizer.step()

        running_train_loss += loss.item()

    with torch.no_grad():
        for batch_features, batch_labels in val_loader:

            batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
            output = model(batch_features)
            _, pred = torch.max(output , 1)

            total += batch_labels.shape[0]
            correct += (pred == batch_labels).sum().item()




    avg_train_loss = running_train_loss/len(train_loader)
    val_accuacry = correct/total
    print(f'Epoch: {epoch+1}, Training Loss: {avg_train_loss:.4f}, Validation Accuracy: {val_accuacry * 100}')



Epoch: 1, Training Loss: 0.0691, Validation Accuracy: 98.53372434017595
Epoch: 2, Training Loss: 0.0689, Validation Accuracy: 98.46041055718476
Epoch: 3, Training Loss: 0.0691, Validation Accuracy: 98.27712609970675
Epoch: 4, Training Loss: 0.0685, Validation Accuracy: 98.60703812316716
Epoch: 5, Training Loss: 0.0657, Validation Accuracy: 98.31378299120234
Epoch: 6, Training Loss: 0.0660, Validation Accuracy: 98.16715542521995
Epoch: 7, Training Loss: 0.0697, Validation Accuracy: 98.71700879765396
Epoch: 8, Training Loss: 0.0675, Validation Accuracy: 98.42375366568915
Epoch: 9, Training Loss: 0.0662, Validation Accuracy: 98.57038123167156
Epoch: 10, Training Loss: 0.0671, Validation Accuracy: 98.53372434017595
Epoch: 11, Training Loss: 0.0672, Validation Accuracy: 98.09384164222874
Epoch: 12, Training Loss: 0.0715, Validation Accuracy: 98.09384164222874
Epoch: 13, Training Loss: 0.0697, Validation Accuracy: 98.71700879765396
Epoch: 14, Training Loss: 0.0703, Validation Accuracy: 98.02

In [68]:
print(test_loader)

In [73]:
model.eval()
total=0
correct=0

with torch.no_grad():

    for batch_features, batch_labels in test_loader:
        batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

        output = model(batch_features)

        _, pred = torch.max(output , 1)

        total += batch_labels.shape[0]
        correct += (pred == batch_labels).sum().item()

accuracy = correct/total
print(f"Accuracy of model on test data: {accuracy:.4f}") 

Accuracy of model on test data: 0.9897
